In [1]:
import os
import shutil
import pommerman
import numpy as np
import tensorflow as tf

from pommerman.agents import BaseAgent, SimpleAgent
from pommerman.configs import ffa_v0_env
from pommerman.constants import BOARD_SIZE
from pommerman.envs.v0 import Pomme
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from keras.utils import to_categorical
# Immitation Learning: learn a mapping from observations to actions.
from pomm_network import PommNetwork

/gpfs/hpchome/anton95/.conda/envs/pommerman/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/gpfs/hpchome/anton95/.conda/envs/pommerman/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
batch_size = 16384
epochs = 25
initial_rollouts = 120
num_rollouts = 50
iterations = 100
early_stopping = 6

In [3]:
class Agent:
    def __init__(self, actions, seed=0, save_path="./model/model.cptk", log_path='./logs/', save_best_model=True, learning_rate=1e-3):
        self.save_path = save_path
        self.actions = actions
        self.save_best_model = save_best_model
        self.seed = seed
        self.sess = tf.InteractiveSession()
        self.rewards = []
        self.cur_epoch = 0
        if os.path.isdir(log_path):
            try:
                os.removedirs(log_path)
            except:
                print("Cant delete log folder")

        # TODO hardcoded
        self.conv_ph = tf.placeholder(shape=[None, BOARD_SIZE, BOARD_SIZE, 3], name='conv_ph', dtype=tf.float32)
        self.state_ph = tf.placeholder(shape=[None, 3], name='state_ph', dtype=tf.float32)
        self.logits_ph = tf.placeholder(shape=[None, actions], name='logits_ph', dtype=tf.int32)

        network = PommNetwork.create_network({'board': self.conv_ph, 'state': self.state_ph})
        logits = tf.layers.dense(network, actions)
        self.sampled_action = tf.squeeze(tf.multinomial(logits, 1), axis=[1])
        sy_logprob_n = tf.nn.softmax_cross_entropy_with_logits(labels=self.logits_ph, logits=logits)
        self.loss = tf.reduce_mean(sy_logprob_n)  # Loss function that we'll differentiate to get the policy gradient.
        self.train_step = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(self.loss)
        self.train_writer = tf.summary.FileWriter(log_path + "train", self.sess.graph)
        self.test_writer = tf.summary.FileWriter(log_path + "test")
        self.sess.run(tf.global_variables_initializer())
        self.saver = tf.train.Saver()
        if os.path.isdir(os.path.dirname(self.save_path)):
            try:
                print("Trying to load model")
                self.saver.restore(self.sess, self.save_path)
                print("Model was loaded successful")
            except:
                print("Model load failed")

    def __run_batch(self, X, y, batch_size, training=True):
        accuracies = []
        losses = []
        size = X.shape[-1]
        batches = int(np.ceil(size / batch_size))
        for i in range(batches):
            X_batch, y_batch = None, None
            if i == batches - 1:
                X_batch = X[i * batch_size:]
                y_batch = y[i * batch_size:]
            else:
                X_batch = X[i * batch_size: (i + 1) * batch_size]
                y_batch = y[i * batch_size: (i + 1) * batch_size]
            board = []
            state = []
            for i in range(X_batch.shape[-1]):
                val = self.featurize(X_batch[i])
                board.append(val['board'])
                state.append(val['state'])
            y_batch = np.array(y_batch).reshape(-1)
            feed_dict = {self.conv_ph: board, self.state_ph: state, self.logits_ph: to_categorical(y_batch, self.actions)}
            if training:
                _, loss, actions = self.sess.run([self.train_step, self.loss, self.sampled_action], feed_dict=feed_dict)
            else:
                loss, actions = self.sess.run([self.loss, self.sampled_action], feed_dict=feed_dict)
            accuracies.append(accuracy_score(actions, y_batch))
            losses.append(loss)
        return np.mean(accuracies), np.mean(losses)

    def train(self, obs, labels, batch_size, epochs, early_stopping = 6):
        print("Train the agent with %i training data, batch_size %i, epochs %i" % (obs.shape[0], batch_size, epochs))
        train_obs, val_obs, train_labels, val_labels = train_test_split(obs, labels, test_size=0.2, random_state=self.seed)
        best_loss = np.inf
        not_improved = 0
        for i in range(epochs):
            train_acc, train_loss = self.__run_batch(train_obs, train_labels, batch_size)
            val_acc, val_loss = self.__run_batch(val_obs, val_labels, batch_size, training=False)
            print("Epoch %d: train_acc %f, train_loss %f, test_acc %f, test_loss: %f" % (i + 1, train_acc, train_loss, val_acc, val_loss))

            summary = tf.summary.Summary()
            summary.value.add(tag="Train acc", simple_value=train_acc)
            summary.value.add(tag="Train loss", simple_value=train_loss)
            self.train_writer.add_summary(summary, self.cur_epoch)
            summary.value.add(tag="Val acc", simple_value=val_acc)
            summary.value.add(tag="Val loss", simple_value=val_loss)
            self.test_writer.add_summary(summary, self.cur_epoch)
            self.cur_epoch += 1
            if not_improved > early_stopping:
                print("Early stopping")
                break
            try:
                if self.save_best_model:
                    if val_loss < best_loss:
                        not_improved = 0
                        print("Saving model")
                        self.saver.save(self.sess, self.save_path)
                        print("Model was saved successfully")
                        best_loss = val_loss
                    else:
                        not_improved += 1
                else:
                    print("Saving model")
                    self.saver.save(self.sess, self.save_path)
                    print("Model was saved successfully")
            except:
                print("Failed save model")

    @staticmethod
    def featurize(obs):
        def get_matrix(dict, key):
            res = dict[key]
            return res.reshape(res.shape[0], res.shape[1], 1).astype(np.float32)

        board = get_matrix(obs, 'board')
        teammate_position = None
        teammate = obs["teammate"]
        if teammate is not None:
            teammate = teammate.value
            if teammate > 10 and teammate < 15:
                teammate_position = np.argwhere(board == teammate)[0]
        else:
            teammate = None
        # My self - 11
        # Team mate - 12
        # Enemy - 13

        # Everyone enemy
        board[(board > 10) & (board < 15)] = 13
        # I'm not enemy
        my_position = obs['position']
        board[my_position[0], my_position[1], 0] = 11
        # Set teammate
        if teammate_position is not None:
            board[teammate_position[0], teammate_position[1], teammate_position[2]] = 12

        bomb_blast_strength = get_matrix(obs, 'bomb_blast_strength')
        bomb_life = get_matrix(obs, 'bomb_life')
        conv_inp = np.concatenate([board, bomb_blast_strength, bomb_life], axis=2)
        state = np.array([obs["ammo"], obs["blast_strength"], obs["can_kick"]]).astype(np.float32)
        return dict(board=conv_inp, state=state)

    def clear_training_history(self):
        self.history = []

    def act(self, obs):
        obs = self.featurize(obs)
        board = np.expand_dims(obs['board'], axis=0)
        state = np.expand_dims(obs['state'], axis=0)
        res = self.sess.run(self.sampled_action, feed_dict={self.conv_ph: board, self.state_ph: state})
        return res

    def record_reward(self, reward):
        self.rewards.append(np.mean(reward))

    def close(self):
        self.sess.close()

In [4]:
# Simple wrapper around policy function to have an act function
class Expert:
    def __init__(self, config):
        self.__agent = SimpleAgent(config)

    def act(self, obs):
        return self.__agent.act(obs, None)

    def record_reward(self, reward):
        pass


class TensorforceAgent(BaseAgent):
    def act(self, obs, action_space):
        pass


# Environment wrapper
class Stimulator:
    def __init__(self, env, config):
        self.env = env
        self.init(config)

    def init(self, config):
        self.env.seed(0)
        # Add 3 random agents
        agents = []
        for agent_id in range(3):
            agents.append(SimpleAgent(config["agent"](agent_id, config["game_type"])))

        # Add TensorforceAgent
        agent_id += 1
        agents.append(TensorforceAgent(config["agent"](agent_id, config["game_type"])))
        self.env.set_agents(agents)
        self.env.set_training_agent(agents[-1].agent_id)
        self.env.set_init_game_state(None)

    def stimulate(self, agent, num_rollouts, render):
        returns = []
        observations = []
        actions = []
        for i in range(num_rollouts):
            obs = self.env.reset()[self.env.training_agent]
            done = False
            totalr = 0.
            steps = 0
            while not done:
                if render:
                    self.env.render()

                action = agent.act(obs)
                obs = self.env.get_observations()
                all_actions = self.env.act(obs)
                all_actions.insert(self.env.training_agent, action)
                state, reward, done, _ = self.env.step(all_actions)

                obs = state[self.env.training_agent]
                r = reward[self.env.training_agent]
                totalr += r
                steps += 1

                observations.append(obs)
                actions.append(action)

            print('rollout %i/%i return=%f' % (i + 1, num_rollouts, totalr))
            returns.append(totalr)
        print('Return summary: mean=%f, std=%f' % (np.mean(returns), np.std(returns)))
        agent.record_reward(returns)
        return (np.array(observations), np.array(actions))

    def label_obs(self, expert, obs):
        actions = []
        for o in obs:
            actions.append(expert.act(o))
        return np.array(actions)

In [5]:
# Instantiate the environment
config = ffa_v0_env()
env = Pomme(**config["env_kwargs"])
states = {
    "board": dict(shape=(BOARD_SIZE, BOARD_SIZE, 3,), type='float'),
    "state": dict(shape=(3,), type='float')
}
agent_dagger = Agent(env.action_space.n)
# Load Expert
expert = Expert(config["agent"](0, config["game_type"]))

# Generate training data
stimulator = Stimulator(env, config)
training_data = stimulator.stimulate(expert, num_rollouts=initial_rollouts, render=False)
# Train DAgger Agent
obs = training_data[0]
labls = training_data[1]
for i in range(2, iterations):
    print("Train with DAgger, iter %i" % i)
    (stimulated_env, _) = stimulator.stimulate(agent_dagger, num_rollouts=num_rollouts, render=False)
    labels = stimulator.label_obs(expert, stimulated_env)
    obs = np.append(obs, stimulated_env, axis=0)
    labls = np.append(labls, labels, axis=0)
    agent_dagger.train(obs, labls, batch_size=batch_size, epochs=epochs, early_stopping=early_stopping)
agent_dagger.close()
env.close()

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
Cant delete log folder
Trying to load model
INFO:tensorflow:Restoring parameters from ./model/model.cptk
Model was loaded successful
rollout 1/120 return=-1.000000
rollout 2/120 return=-1.000000
rollout 3/120 return=-1.000000
rollout 4/120 return=-1.000000
rollout 5/120 return=-1.000000
rollout 6/120 return=-1.000000
rollout 7/120 return=-1.000000
rollout 8/120 return=-1.000000
rollout 9/120 return=1.000000
rollout 10/120 return=-1.000000
rollout 11/120 return=1.000000
rollout 12/120 return=-1.000000
rollout 13/120 return=-1.000000
rollout 14/120 return=-1.000000
rollout 15/120 return=-1.000000
rollout 16/120 return=-1.000000
rollout 17/120 return=-1.000000
rollout 18/120 return=-1.000000
rollout 19/120 return=-1.000000
rollout 20/120 return=1.000000
rollout 21/120 return=-1.000000
rollout 22/120 return=-1.000000
rollout 23/120 return=-1.000000
rollout 24/120 return=-1.000000
rollout 25/1

rollout 2/50 return=-1.000000
rollout 3/50 return=-1.000000
rollout 4/50 return=-1.000000
rollout 5/50 return=-1.000000
rollout 6/50 return=-1.000000
rollout 7/50 return=-1.000000
rollout 8/50 return=-1.000000
rollout 9/50 return=-1.000000
rollout 10/50 return=-1.000000
rollout 11/50 return=-1.000000
rollout 12/50 return=-1.000000
rollout 13/50 return=-1.000000
rollout 14/50 return=-1.000000
rollout 15/50 return=-1.000000
rollout 16/50 return=-1.000000
rollout 17/50 return=-1.000000
rollout 18/50 return=-1.000000
rollout 19/50 return=-1.000000
rollout 20/50 return=-1.000000
rollout 21/50 return=-1.000000
rollout 22/50 return=-1.000000
rollout 23/50 return=-1.000000
rollout 24/50 return=-1.000000
rollout 25/50 return=-1.000000
rollout 26/50 return=-1.000000
rollout 27/50 return=-1.000000
rollout 28/50 return=-1.000000
rollout 29/50 return=-1.000000
rollout 30/50 return=-1.000000
rollout 31/50 return=-1.000000
rollout 32/50 return=-1.000000
rollout 33/50 return=-1.000000
rollout 34/50 re

rollout 4/50 return=-1.000000
rollout 5/50 return=-1.000000
rollout 6/50 return=-1.000000
rollout 7/50 return=-1.000000
rollout 8/50 return=-1.000000
rollout 9/50 return=-1.000000
rollout 10/50 return=-1.000000
rollout 11/50 return=-1.000000
rollout 12/50 return=-1.000000
rollout 13/50 return=-1.000000
rollout 14/50 return=-1.000000
rollout 15/50 return=-1.000000
rollout 16/50 return=-1.000000
rollout 17/50 return=-1.000000
rollout 18/50 return=-1.000000
rollout 19/50 return=-1.000000
rollout 20/50 return=-1.000000
rollout 21/50 return=-1.000000
rollout 22/50 return=-1.000000
rollout 23/50 return=-1.000000
rollout 24/50 return=-1.000000
rollout 25/50 return=-1.000000
rollout 26/50 return=-1.000000
rollout 27/50 return=-1.000000
rollout 28/50 return=-1.000000
rollout 29/50 return=-1.000000
rollout 30/50 return=-1.000000
rollout 31/50 return=-1.000000
rollout 32/50 return=-1.000000
rollout 33/50 return=-1.000000
rollout 34/50 return=-1.000000
rollout 35/50 return=-1.000000
rollout 36/50 

rollout 15/50 return=-1.000000
rollout 16/50 return=-1.000000
rollout 17/50 return=-1.000000
rollout 18/50 return=-1.000000
rollout 19/50 return=-1.000000
rollout 20/50 return=-1.000000
rollout 21/50 return=-1.000000
rollout 22/50 return=-1.000000
rollout 23/50 return=-1.000000
rollout 24/50 return=-1.000000
rollout 25/50 return=-1.000000
rollout 26/50 return=-1.000000
rollout 27/50 return=-1.000000
rollout 28/50 return=-1.000000
rollout 29/50 return=-1.000000
rollout 30/50 return=-1.000000
rollout 31/50 return=-1.000000
rollout 32/50 return=-1.000000
rollout 33/50 return=-1.000000
rollout 34/50 return=-1.000000
rollout 35/50 return=-1.000000
rollout 36/50 return=-1.000000
rollout 37/50 return=-1.000000
rollout 38/50 return=-1.000000
rollout 39/50 return=-1.000000
rollout 40/50 return=-1.000000
rollout 41/50 return=-1.000000
rollout 42/50 return=-1.000000
rollout 43/50 return=-1.000000
rollout 44/50 return=-1.000000
rollout 45/50 return=-1.000000
rollout 46/50 return=-1.000000
rollout 

Train the agent with 109733 training data, batch_size 16384, epochs 25
Epoch 1: train_acc 0.822177, train_loss 0.389930, test_acc 0.774444, test_loss: 0.556122
Saving model
Model was saved successfully
Epoch 2: train_acc 0.826417, train_loss 0.314726, test_acc 0.772136, test_loss: 0.552621
Saving model
Model was saved successfully
Epoch 3: train_acc 0.833752, train_loss 0.291308, test_acc 0.781459, test_loss: 0.552148
Saving model
Model was saved successfully
Epoch 4: train_acc 0.849002, train_loss 0.264888, test_acc 0.776272, test_loss: 0.558774
Epoch 5: train_acc 0.855233, train_loss 0.250406, test_acc 0.781209, test_loss: 0.588486
Epoch 6: train_acc 0.858694, train_loss 0.245655, test_acc 0.784304, test_loss: 0.591841
Epoch 7: train_acc 0.863253, train_loss 0.235710, test_acc 0.774827, test_loss: 0.631118
Epoch 8: train_acc 0.862542, train_loss 0.237524, test_acc 0.776620, test_loss: 0.643035
Epoch 9: train_acc 0.862136, train_loss 0.240962, test_acc 0.777635, test_loss: 0.645302
Ep

...

{(0, 5): None, (0, 8): None, (0, 9): None, (0, 11): None, (1, 4): None, (1, 5): None, (1, 6): None, (1, 7): None, (1, 8): None, (1, 9): None, (1, 10): None, (1, 11): None, (1, 12): None, (2, 5): None, (2, 6): None, (2, 7): None, (2, 11): None, (2, 12): None, (3, 6): None, (3, 8): None, (3, 10): None, (3, 11): None, (4, 7): None, (4, 9): None, (5, 8): None, (5, 9): None, (5, 10): None, (5, 11): None, (6, 10): None, (6, 11): None, (7, 10): None, (7, 11): None, (7, 12): None, (8, 11): None} None (1, 12)
{(0, 5): None, (0, 8): None, (0, 9): None, (0, 11): None, (1, 4): None, (1, 5): None, (1, 6): None, (1, 7): None, (1, 8): None, (1, 9): None, (1, 10): None, (1, 11): None, (1, 12): None, (2, 5): None, (2, 6): None, (2, 7): None, (2, 11): None, (2, 12): None, (3, 6): None, (3, 8): None, (3, 10): None, (3, 11): None, (4, 7): None, (4, 9): None, (5, 8): None, (5, 9): None, (5, 10): None, (5, 11): None, (6, 10): None, (6, 11): None, (7, 10): None, (7, 11): None, (7, 12): None, (8, 11): None} N

rollout 39/50 return=-1.000000
rollout 40/50 return=-1.000000
rollout 41/50 return=-1.000000
rollout 42/50 return=-1.000000
rollout 43/50 return=-1.000000
rollout 44/50 return=-1.000000
rollout 45/50 return=-1.000000
rollout 46/50 return=-1.000000
rollout 47/50 return=-1.000000
rollout 48/50 return=-1.000000
rollout 49/50 return=-1.000000
rollout 50/50 return=-1.000000
Return summary: mean=-1.000000, std=0.000000
Train the agent with 408051 training data, batch_size 16384, epochs 25
Epoch 1: train_acc 0.959508, train_loss 0.076433, test_acc 0.932398, test_loss: 0.263712
Saving model
Model was saved successfully
Epoch 2: train_acc 0.962832, train_loss 0.061293, test_acc 0.933000, test_loss: 0.260278
Saving model
Model was saved successfully
Epoch 3: train_acc 0.964836, train_loss 0.057105, test_acc 0.932716, test_loss: 0.276915
Epoch 4: train_acc 0.965131, train_loss 0.056499, test_acc 0.932785, test_loss: 0.278816
Epoch 5: train_acc 0.965859, train_loss 0.055586, test_acc 0.932795, tes

{(0, 2): None, (0, 5): None, (0, 7): None, (0, 9): None, (0, 10): None, (0, 11): None, (0, 12): None, (1, 2): None, (1, 3): None, (1, 4): None, (1, 5): None, (1, 12): None, (2, 4): None, (2, 5): None, (2, 6): None, (2, 7): None, (2, 11): None, (2, 12): None, (3, 4): None, (3, 6): None, (3, 10): None, (3, 11): None, (3, 12): None, (4, 6): None, (4, 8): None, (4, 9): None, (4, 10): None, (4, 11): None, (4, 12): None, (5, 8): None, (5, 10): None, (5, 11): None, (6, 9): None, (6, 11): None, (7, 9): None, (7, 10): None} None (0, 9)
Train the agent with 414778 training data, batch_size 16384, epochs 25
Epoch 1: train_acc 0.962125, train_loss 0.068788, test_acc 0.933029, test_loss: 0.256902
Saving model
Model was saved successfully
Epoch 2: train_acc 0.963906, train_loss 0.058576, test_acc 0.933017, test_loss: 0.263906
Epoch 3: train_acc 0.965452, train_loss 0.057608, test_acc 0.932355, test_loss: 0.262248
Epoch 4: train_acc 0.965628, train_loss 0.055785, test_acc 0.932752, test_loss: 0.27131

rollout 46/50 return=-1.000000
rollout 47/50 return=-1.000000
rollout 48/50 return=-1.000000
rollout 49/50 return=-1.000000
rollout 50/50 return=-1.000000
Return summary: mean=-1.000000, std=0.000000
Train the agent with 425180 training data, batch_size 16384, epochs 25
Epoch 1: train_acc 0.960941, train_loss 0.073698, test_acc 0.931291, test_loss: 0.252828
Saving model
Model was saved successfully
Epoch 2: train_acc 0.963949, train_loss 0.059308, test_acc 0.933449, test_loss: 0.258159
Epoch 3: train_acc 0.964818, train_loss 0.056762, test_acc 0.934892, test_loss: 0.261044
Epoch 4: train_acc 0.965829, train_loss 0.055735, test_acc 0.934797, test_loss: 0.285277
Epoch 5: train_acc 0.965820, train_loss 0.056150, test_acc 0.933907, test_loss: 0.309413
Epoch 6: train_acc 0.965860, train_loss 0.056028, test_acc 0.933920, test_loss: 0.307857
Epoch 7: train_acc 0.965560, train_loss 0.056609, test_acc 0.933971, test_loss: 0.294894
Epoch 8: train_acc 0.966399, train_loss 0.055852, test_acc 0.933

Train the agent with 429139 training data, batch_size 16384, epochs 25
Epoch 1: train_acc 0.960105, train_loss 0.077261, test_acc 0.933663, test_loss: 0.252931
Saving model
Model was saved successfully
Epoch 2: train_acc 0.964049, train_loss 0.059286, test_acc 0.935266, test_loss: 0.261972
Epoch 3: train_acc 0.965888, train_loss 0.055792, test_acc 0.935830, test_loss: 0.276925
Epoch 4: train_acc 0.966213, train_loss 0.054622, test_acc 0.934887, test_loss: 0.280509
Epoch 5: train_acc 0.966973, train_loss 0.054519, test_acc 0.935109, test_loss: 0.285005
Epoch 6: train_acc 0.966527, train_loss 0.054878, test_acc 0.934838, test_loss: 0.285556
Epoch 7: train_acc 0.966327, train_loss 0.055554, test_acc 0.932793, test_loss: 0.297169
Epoch 8: train_acc 0.965183, train_loss 0.057797, test_acc 0.933611, test_loss: 0.278429
Epoch 9: train_acc 0.964511, train_loss 0.058924, test_acc 0.932595, test_loss: 0.279724
Early stopping
Train with DAgger, iter 99
rollout 1/50 return=-1.000000
rollout 2/50 r